In [ ]:
%load_ext dotenv

# Use find_dotenv to locate the file
%dotenv

import os

In [ ]:
import pandas as pd
import json
import requests
import datetime

In [ ]:
transcripts_folder = "company_transcripts/"
mkdata_folder = "mkdata/"

In [ ]:
import time
from pathlib import Path

from os import listdir
from os.path import isfile, join
transcripts_files = [f for f in listdir(transcripts_folder) if isfile(join(transcripts_folder, f)) and f.split(".")[-1] == 'json']

from os import listdir
from os.path import isfile, join
mkdata_files = [f for f in listdir(mkdata_folder) if isfile(join(mkdata_folder, f)) and f.split(".")[-1] == 'json']


for transcripts_file in [item for item in transcripts_files if item not in mkdata_files]:
    file = Path(f'{transcripts_folder}{transcripts_file}')
    if file.stat().st_size >0:
        tmp_symbol = transcripts_file.split(".")[0]
        
        df = alphavantage_timeSeriesDailyAdjusted(tmp_symbol)
        df.to_json(f'{mkdata_folder}{transcripts_file}', orient='records')
        
        time.sleep(30)
        #break
        print(tmp_symbol)

In [ ]:
import json
import pandas as pd
import datetime
import time

def alphavantage_timeSeriesDailyAdjusted(symbol):
    res = requests.get(f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&outputsize=full&apikey={os.getenv("ALPHAVANTAGE_API")}')
    
    max_retry = 5
    k = 0
    while 'Meta' not in res.text:
        print(res.content)
        time.sleep(60)
        res = requests.get(f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&outputsize=full&apikey={os.getenv("ALPHAVANTAGE_API")}')
        
        k += 1
        
        if k >= max_retry:
            print('unable to use API')
            break
        
    
    
    dict_data = json.loads(res.content)
    
    for tmp_field in dict_data.keys():
        if 'Meta' in tmp_field:
            hdr_metadata = tmp_field
        elif 'Time' in tmp_field and 'Serie' in tmp_field:
            hdr_ts = tmp_field
    
        
        
    for tmp_field in dict_data[hdr_metadata].keys():
        if 'Symbol' in tmp_field:
            hdr_symbol = tmp_field
            break
    
    tmp_symbol = dict_data[hdr_metadata][hdr_symbol]
    
    for date_tmp in dict_data[hdr_ts]:
        data_dim = dict_data[hdr_ts][date_tmp].keys()
        break
    
    for tmp_key in data_dim:
        if 'open' in tmp_key:
            hdr_open = tmp_key
        elif 'high' in tmp_key:
            hdr_high = tmp_key
        elif 'low' in tmp_key:
            hdr_low = tmp_key
        elif 'close' in tmp_key and 'adjusted' not in tmp_key:
            hdr_close = tmp_key
        elif 'adjusted close' in tmp_key:
            hdr_adjclose = tmp_key
        elif 'volume' in tmp_key:
            hdr_volume = tmp_key
        elif 'dividend' in tmp_key:
            hdr_dividend = tmp_key
        elif 'split' in tmp_key:
            hdr_split = tmp_key
    
    list_data = []

    bridge = {
        'open': hdr_open,
        'high': hdr_high,
        'low': hdr_low,
        'close': hdr_close,
        'adjclose': hdr_adjclose,
        'volume': hdr_volume,
        'div': hdr_dividend,
        'split': hdr_split,
    }

    for tmp_date in dict_data[hdr_ts]:

        if tmp_date != datetime.date.today().strftime('%Y-%m-%d'):

            row = {
                'symbol': tmp_symbol,
                'dateeod': tmp_date,
            }

            for tmp_conv_num in bridge:
                try:
                    row[tmp_conv_num] = float(dict_data[hdr_ts][tmp_date][bridge[tmp_conv_num]])
                except:
                    row[tmp_conv_num] = None


            list_data.append( row )
    
    df = pd.DataFrame(list_data)
    
    return df
    
    